<a href="https://colab.research.google.com/github/PrasanthBalaraman/MachineLearning/blob/master/SmartSummarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim

    100% |████████████████████████████████| 23.6MB 1.4MB/s 
    100% |████████████████████████████████| 1.4MB 14.1MB/s 
    100% |████████████████████████████████| 133kB 29.2MB/s 
    100% |████████████████████████████████| 61kB 23.1MB/s 
    100% |████████████████████████████████| 5.0MB 6.7MB/s 
    100% |████████████████████████████████| 552kB 21.9MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [2]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk.tokenize import word_tokenize
import re
import collections
import pickle
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

train_article_path ="C:\\Users\\muvaiz\\Documents\\Genesys_Demojam\\Code\\sample_data\\sumdata\\train\\train.article.txt"
train_title_path ="C:\\Users\\muvaiz\\Documents\\Genesys_Demojam\\Code\\sample_data\\sumdata\\train\\train.title.txt"
#valid_article_path ="C:\\Users\\muvaiz\\Documents\\Genesys_Demojam\\Code\\sample_data\\sumdata\\train\\valid.article.filter.txt"
#valid_title_path ="C:\\Users\\muvaiz\\Documents\\Genesys_Demojam\\Code\\sample_data\\sumdata\\train\\valid.title.filter.txt"
valid_article_path ="C:\\Users\\muvaiz\\Documents\\Genesys_Demojam\\Code\\sample_data\\sumdata\\train\\test.article.txt"

#to clean sentence
def clean_str(sentence):
    sentence=re.sub("[#.]+","#",sentence)
    return sentence

#read each line from given file and will pass it to clean_str function for cleaning sentence. "toy" flag will be used for optimization
def get_text_list(data_path,toy):
    with open(data_path,'r',encoding="utf-8") as f:
        if not toy:
            return([clean_str(x.strip()) for x in f.readlines()])
        else:
            return([clean_str(x.strip()) for x in f.readlines()][:50000])
      
def batch_iter(inputs, outputs, batch_size, num_epochs):
    inputs = np.array(inputs)
    outputs = np.array(outputs)

    num_batches_per_epoch = (len(inputs) - 1) // batch_size + 1
    for epoch in range(num_epochs):
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, len(inputs))
            yield inputs[start_index:end_index], outputs[start_index:end_index]
            
def get_init_embedding(reversed_dict,embedding_size):
    print("Using GloVe")
    glove_file= 'glove.6B.200d.txt'  #"C:\\Users\\muvaiz\\Documents\\Python_Scripts\\LSTM\\glove.6B\\glove.6B.50d.txt"
    word2vec_file=get_tmpfile("word2vec_format.vec")
    glove2word2vec(glove_file,word2vec_file)
    print("Loading Glove vectors...")
    word_vectors = KeyedVectors.load_word2vec_format(word2vec_file)
    
    word_vec_list = list()
    
    for _,word in sorted(reversed_dict.items()):
        try:
            word_vec = word_vectors.word_vec(word)
        except KeyError:
            
            word_vec=np.zeros([embedding_size],dtype=np.float32)
            
        word_vec_list.append(word_vec)
        
    word_vec_list[2] = np.random.normal(0, 1, embedding_size)
    word_vec_list[3] = np.random.normal(0, 1, embedding_size)
    
    return(np.array(word_vec_list))

In [8]:
from nltk.tokenize import word_tokenize
import re
import collections
import pickle
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

# load from file
stories = pickle.load(open('cnn_dataset.pkl', 'rb'))
print('Loaded Stories %d' % len(stories))

def print_story(stories):
  index = np.random.randint(90000)
  story, highlights = stories[index]['story'], stories[index]['highlights']
  print("\nSample Story\n")
  print("Story: \n")
  print(story)
  print("Highlights: \n")
  print(highlights)
  print("\n")
print_story(stories)

def build_dict(step,stories,toy=False):
    if step=="train":
        #train_article_list=get_text_list(train_article_path,toy)
        #train_title_list=get_text_list(train_title_path,toy)
        train_article_list=list()
        train_title_list=list()
        for each_story_highlight in stories:
            train_article_list.append(each_story_highlight['story'])
            train_title_list.append(each_story_highlight['highlights'])
        
        words=list()
        
        for paragraph in train_article_list:
            for sentence in paragraph:
                for word in word_tokenize(sentence):
                    words.append(word)
                
        word_counter=collections.Counter(words).most_common()
        word_dict=dict()
        word_dict["<padding>"]=0
        word_dict["<unk>"]=1
        word_dict["<s>"]=2
        word_dict["</s>"]=3
        
        for word,_ in word_counter:
            word_dict[word]=len(word_dict)
            
        with open("word_dict.pickle","wb") as f:
            pickle.dump(word_dict,f)
            
    elif step=="valid":
        with open("word_dict.pickle","rb") as f:
            word_dict=pickle.load(f)
            
    reversed_dict = dict(zip(word_dict.values(),word_dict.keys()))
    
    article_max_len=50
    summary_max_len=15
    
    return word_dict,reversed_dict,article_max_len,summary_max_len
  

def build_dataset(step,stories,word_dict,article_max_len,summary_max_len,toy=False):
    if step=="train":
        article_list=list()
        title_list=list()
        for each_story_highlight in stories:
            article_list.append(each_story_highlight['story'])
            train_title_list.append(each_story_highlight['highlights'])
        article_list=get_text_list(train_article_path,toy)
        title_list=get_text_list(train_title_path,toy)
    elif step=="valid":
        article_list=get_text_list(valid_article_path,toy)
    else:
        raise NotImplementedError
    
    #sample output of sentence_with_word_lists_per_sent : [["I","am","a","boy"],["I","like","rain"]]
    sentences_with_word_lists_per_sent=[list(map(word_tokenize, paragraph)) for paragraph in article_list]
    
    #sample output of sentence_with_word_lists_per_sent : [["101","21","343","711"],["100","234","1526"]]
    sentences_with_word_vector_lists_per_sent=[[word_dict.get(word,word_dict["<unk>"]) for word in sentence] for sentence in sentences_with_word_lists_per_sent]
                                                              
    truncated_word_vector_lists_per_sent=[sentence[:article_max_len] for sentence in sentences_with_word_vector_lists_per_sent]
    output_word_vector_lists=[sentence + (article_max_len - len(sentence)) * [word_dict["<padding>"]] for sentence in truncated_word_vector_lists_per_sent]
                                                              
    if step=="valid":
        return output_word_vector_lists
    else:
        title_with_word_lists_per_sent = [word_tokenize(sentence) for sentence in title_list]
        title_with_word_vector_lists_per_sent=[[word_dict.get(word,word_dict["<unk>"]) for word in sentence] for sentence in title_with_word_lists_per_sent]
        output_title_vector_lists=[sentence[:(summary_max_len-1)]for sentence in title_with_word_vector_lists_per_sent]
        return(output_word_vector_lists,output_title_vector_lists)


Loaded Stories 92579

Sample Story

Story: 

['hes fluent in both english and arabic and if his college degree in computers is any indication he has a way with technology then theres his interest in radical islam', 'put it all together and authorities say former boston resident and us citizen ahmad abousamra could be a good fit inside the isis social media machine thats become renowned in recent weeks for spewing brutal propaganda across social media messages meant both to terrify and recruit westerners', 'theres no specific evidence pointing to abousamras involvement in the terror groups social media campaign but federal investigators think he may have joined the group and is helping with online efforts a law enforcement official told cnn', 'abousamra who holds dual us and syrian citizenship is wanted by the fbi on terrorism charges issued in they include providing material support to terrorists theres a reward for his capture', 'according to the fbi abousamra repeatedly traveled to p

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 26 23:47:16 2018

@author: muvaiz
"""

import time
start = time.perf_counter()
import tensorflow as tf
import argparse
import pickle
import os
#from model import Model
#from utils import build_dict, build_dataset, batch_iter

class Simulate_Arguments():
  
  def __init__(self):
    self.num_hidden=150
    self.num_layers=2
    self.beam_width=10
    self.glove=True
    self.embedding_size=200
    
    self.learning_rate=1e-3
    self.batch_size=64
    self.num_epochs=10
    self.keep_prob=0.8
    
    self.toy=True
    self.with_model=False

# Uncomment next 2 lines to suppress error and Tensorflow info verbosity. Or change logging levels
# tf.logging.set_verbosity(tf.logging.FATAL)
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# def add_arguments(parser):
#     parser.add_argument("--num_hidden", type=int, default=150, help="Network size.")
#     parser.add_argument("--num_layers", type=int, default=2, help="Network depth.")
#     parser.add_argument("--beam_width", type=int, default=10, help="Beam width for beam search decoder.")
#     parser.add_argument("--glove", action="store_true", help="Use glove as initial word embedding.")
#     parser.add_argument("--embedding_size", type=int, default=300, help="Word embedding size.")

#     parser.add_argument("--learning_rate", type=float, default=1e-3, help="Learning rate.")
#     parser.add_argument("--batch_size", type=int, default=64, help="Batch size.")
#     parser.add_argument("--num_epochs", type=int, default=1, help="Number of epochs.")
#     parser.add_argument("--keep_prob", type=float, default=0.8, help="Dropout keep prob.")

#     parser.add_argument("--toy", action="store_true", help="Use only 50K samples of data")

#     parser.add_argument("--with_model", action="store_true", help="Continue from previously saved model")



#parser = argparse.ArgumentParser()
#add_arguments(parser)
args = Simulate_Arguments()
with open("args.pickle", "wb") as f:
    pickle.dump(args, f)

if not os.path.exists("saved_model"):
    os.mkdir("saved_model")
else:
    if args.with_model:
        old_model_checkpoint_path = open('saved_model/checkpoint', 'r')
        old_model_checkpoint_path = "".join(["saved_model/",old_model_checkpoint_path.read().splitlines()[0].split('"')[1] ])

# load from file
stories = pickle.load(open('cnn_dataset.pkl', 'rb'))
print('Loaded Stories %d' % len(stories))
        
print("Building dictionary...")
word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("train",stories, args.toy)
print("Loading training dataset...")
train_x, train_y = build_dataset("train", stories, word_dict, article_max_len, summary_max_len, args.toy)


with tf.Session() as sess:
    model = Model(reversed_dict, article_max_len, summary_max_len, args)
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(tf.global_variables())
    if 'old_model_checkpoint_path' in globals():
        print("Continuing from previous trained model:" , old_model_checkpoint_path , "...")
        saver.restore(sess, old_model_checkpoint_path )

    batches = batch_iter(train_x, train_y, args.batch_size, args.num_epochs)
    num_batches_per_epoch = (len(train_x) - 1) // args.batch_size + 1

    print("\nIteration starts.")
    print("Number of batches per epoch :", num_batches_per_epoch)
    for batch_x, batch_y in batches:
        batch_x_len = list(map(lambda x: len([y for y in x if y != 0]), batch_x))
        batch_decoder_input = list(map(lambda x: [word_dict["<s>"]] + list(x), batch_y))
        batch_decoder_len = list(map(lambda x: len([y for y in x if y != 0]), batch_decoder_input))
        batch_decoder_output = list(map(lambda x: list(x) + [word_dict["</s>"]], batch_y))

        batch_decoder_input = list(
            map(lambda d: d + (summary_max_len - len(d)) * [word_dict["<padding>"]], batch_decoder_input))
        batch_decoder_output = list(
            map(lambda d: d + (summary_max_len - len(d)) * [word_dict["<padding>"]], batch_decoder_output))

        train_feed_dict = {
            model.batch_size: len(batch_x),
            model.X: batch_x,
            model.X_len: batch_x_len,
            model.decoder_input: batch_decoder_input,
            model.decoder_len: batch_decoder_len,
            model.decoder_target: batch_decoder_output
        }

        _, step, loss = sess.run([model.update, model.global_step, model.loss], feed_dict=train_feed_dict)

        if step % 1000 == 0:
            print("step {0}: loss = {1}".format(step, loss))

        if step % num_batches_per_epoch == 0:
            hours, rem = divmod(time.perf_counter() - start, 3600)
            minutes, seconds = divmod(rem, 60)
            saver.save(sess, "./saved_model/model.ckpt", global_step=step)
            print(" Epoch {0}: Model is saved.".format(step // num_batches_per_epoch),
            "Elapsed: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds) , "\n")

Loaded Stories 92579
Building dictionary...
